In [8]:
import tkinter as tk
from tkinter import messagebox
import random

# Player2 class from code 1
class Player2:
    def __init__(self):
        self.deflect_counter = 0
        self.cooperate_counter = 0
        self.personalities = {
            "Kind Person": self.kind_person,
            "Rude Person": self.rude_person,
            "Logical Person": self.logical_person,
            "Religious Person": self.religious_person,
            "Opposite Person": self.opposite_person
        }
        self.used_personalities = []  # Keep track of used personalities
        self.current_personality = None
        self.previous_action = 'D'  # Initially set to deflect

    def kind_person(self):
        if self.deflect_counter >= 10:
            return 'D'
        else:
            return 'C'

    def rude_person(self):
        if self.cooperate_counter >= 12:
            return 'C'
        else:
            return 'D'

    def logical_person(self):
        return 'D'

    def religious_person(self):
        if self.deflect_counter >= 23:
            return 'D'
        else:
            return 'C'

    def opposite_person(self):
        return 'C' if self.previous_action == 'D' else 'D'

    def choose_personality(self, available_personalities):
        unused_personalities = [p for p in available_personalities.keys() if p not in self.used_personalities]
        if unused_personalities:
            personality_name = random.choice(unused_personalities)
            self.current_personality = available_personalities[personality_name]
            self.used_personalities.append(personality_name)
            return personality_name
        return None

    def choose_action(self):
        return self.current_personality()

class PrisonersDilemma:
    PAYOFF_MATRIX = {
        ("C", "C"): (20, 20),
        ("C", "D"): (0, 30),
        ("D", "C"): (30, 0),
        ("D", "D"): (10, 10)
    }

    def __init__(self, master):
        self.master = master
        master.title("Repeated Prisoner's Dilemma")

        self.player2 = Player2()
        self.round_count = 0
        self.total_rounds = 25
        self.your_scores = []
        self.opponent_scores = []
        self.current_personality_name = ""

        # Display the payoff matrix
        self.matrix_frame = tk.Frame(master)
        self.matrix_frame.grid(row=0, column=0, padx=20, pady=20)

        self.matrix_label = tk.Label(
            self.matrix_frame,
            text="        Cooperate  Defect\n"
                 "Coop   (20, 20)      (0, 30)\n"
                 "Defect (30, 0)        (10, 10)",
            font=("Arial", 14),
            justify="left"
        )
        self.matrix_label.pack()

        # Display average payoffs and opponent personality
        self.personality_label = tk.Label(master, text="Personality: ", font=("Arial", 10))
        self.personality_label.grid(row=1, column=0, padx=2, pady=2)

        self.your_average_label = tk.Label(master, text="Your Average Payoff: 0.0", font=("Arial", 10))
        self.your_average_label.grid(row=2, column=0, padx=2, pady=2)

        self.opponent_average_label = tk.Label(master, text="Opponent Average Payoff: 0.0", font=("Arial", 10))
        self.opponent_average_label.grid(row=3, column=0, padx=2, pady=2)

        # Create table frame for displaying round-by-round payoffs
        self.table_frame = tk.Frame(master)
        self.table_frame.grid(row=4, column=0, padx=10, pady=10)

        # Display round-by-round payoff table headers
        self.headers = ["Round", "Your Payoff", "Opponent's Payoff"]
        for col, header in enumerate(self.headers):
            label = tk.Label(self.table_frame, text=header, font=("Arial", 10))
            label.grid(row=0, column=col, padx=5, pady=2)

        # Buttons for Cooperate and Defect
        self.cooperate_button = tk.Button(master, text="Cooperate", width=10, background="green", fg="white", command=lambda: self.play_round("C"))
        self.cooperate_button.grid(row=5, column=0, padx=10, pady=10)

        self.defect_button = tk.Button(master, text="Defect", width=10, background="red", fg="white", command=lambda: self.play_round("D"))
        self.defect_button.grid(row=5, column=1, padx=10, pady=10)

    def update_averages(self):
        if self.your_scores:
            your_average = sum(self.your_scores) / len(self.your_scores)
        else:
            your_average = 0

        if self.opponent_scores:
            opponent_average = sum(self.opponent_scores) / len(self.opponent_scores)
        else:
            opponent_average = 0

        self.your_average_label.config(text=f"Your Average Payoff: {your_average:.1f}")
        self.opponent_average_label.config(text=f"Opponent Average Payoff: {opponent_average:.1f}")

    def clear_table(self):
        # Clear the table by removing all widgets in the table_frame
        for widget in self.table_frame.winfo_children():
            widget.destroy()
        # Redisplay headers
        for col, header in enumerate(self.headers):
            label = tk.Label(self.table_frame, text=header, font=("Arial", 10))
            label.grid(row=0, column=col, padx=5, pady=2)

    def play_round(self, your_move):
        if self.round_count == 0:
            self.clear_table()
            self.current_personality_name = self.player2.choose_personality(self.player2.personalities)
            self.personality_label.config(text=f"Personality: {self.current_personality_name}")

        opponent_move = self.player2.choose_action()

        if your_move == 'C':
            self.player2.cooperate_counter = 0
        elif your_move == 'D':
            self.player2.deflect_counter += 1
            self.player2.cooperate_counter += 1

        your_payoff, opponent_payoff = self.PAYOFF_MATRIX[(your_move, opponent_move)]

        self.your_scores.append(your_payoff)
        self.opponent_scores.append(opponent_payoff)
        self.update_averages()

        # Update round-by-round payoff table
        row_num = self.round_count % 12 + 1  # Start from row 1 for each set of 12 rounds
        col_offset = (self.round_count // 12) * 3  # Create new columns every 12 rounds

        round_label = tk.Label(self.table_frame, text=self.round_count + 1, font=("Arial", 10))
        round_label.grid(row=row_num, column=0 + col_offset, padx=5, pady=2)

        your_payoff_label = tk.Label(self.table_frame, text=your_payoff, font=("Arial", 10))
        your_payoff_label.grid(row=row_num, column=1 + col_offset, padx=5, pady=2)

        opponent_payoff_label = tk.Label(self.table_frame, text=opponent_payoff, font=("Arial", 10))
        opponent_payoff_label.grid(row=row_num, column=2 + col_offset, padx=5, pady=2)

        self.player2.previous_action = opponent_move

        # Check if all rounds against current opponent are complete
        self.round_count += 1
        if self.round_count == self.total_rounds:
            messagebox.showinfo("Round Complete", f"Finished rounds against opponent with personality {self.current_personality_name}")
            self.calculate_average_payoffs()

            self.round_count = 0
            self.your_scores = []
            self.opponent_scores = []

            if not self.player2.personalities:
                messagebox.showinfo("Game Over", "All games complete!")
                self.show_summary()
            else:
                self.player2 = Player2()  # Reset Player2 for the next game
                messagebox.showinfo("Next Game", "Starting new game with a fresh set of personalities")
                self.personality_label.config(text="Personality: ")

    def calculate_average_payoffs(self):
        your_average = sum(self.your_scores) / len(self.your_scores) if self.your_scores else 0
        opponent_average = sum(self.opponent_scores) / len(self.opponent_scores) if self.opponent_scores else 0
        messagebox.showinfo("Final Average Payoffs", f"Your Average Payoff: {your_average:.1f}\nOpponent Average Payoff: {opponent_average:.1f}")

    def show_summary(self):
        summary = "Summary of Average Payoffs:\n\n"
        # No need to loop through OPPONENTS, as Player2 handles personality internally
        your_avg = sum(self.your_scores) / len(self.your_scores) if self.your_scores else 0
        opponent_avg = sum(self.opponent_scores) / len(self.opponent_scores) if self.opponent_scores else 0
        summary += f"Your Average Payoff: {your_avg:.1f}, Opponent Average Payoff: {opponent_avg:.1f}\n"
        messagebox.showinfo("Summary", summary)


if __name__ == "__main__":
    root = tk.Tk()
    game = PrisonersDilemma(root)
    root.mainloop()